In [ ]:

#######################################################################################
                                # Load packages #
#######################################################################################

import os
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy.interpolate import interp2d
from scipy.signal import find_peaks

from IPython.display import display
from ipyfilechooser import FileChooser

import pickle
import os
from scipy.interpolate import griddata
import logging
import sys 
import shutil
from bisect import bisect_left
from ast import literal_eval
import fnmatch


from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer
from IPython.display import display
from ipyfilechooser import FileChooser
from datetime import datetime


In [ ]:
#######################################################################################
                                # Define functions #
#######################################################################################

def take_closest2(myList, myNumber):
    value2 = 10000000
    for ind in range(len(myList)):
        value = abs(myList[ind]-myNumber)
        if value < value2:
            value2 = value
            index = myList[ind]
    return index


# Function to find files containing a specific string
def find_files_with_string(folder_path, search_string):
    matching_file = []
    # Traverse the folder to find files
    for root, _, files in os.walk(folder_path):
        for file in files:
            if fnmatch.fnmatch(file, f"*{search_string}*"):
                matching_file=os.path.join(root, file)
    return matching_file


In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with Excel Files</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

In [ ]:
folderpath = Path(dpath)

List= find_files_with_string(folderpath, "Spindlespropreties_")
for file in List:
    if 'PFCInitial' in file:
        listSpdl  = pd.read_excel(file)
        Spdlfile=file
        print('Spdl file:', file)

List= find_files_with_string(folderpath, "SWRpropreties_")
for file in List:
    if 'CA1Initial' in file:
        listSWR  = pd.read_excel(file)
        SWRfile=file
        print('SWR file:', file)

In [ ]:
#######################################################################################
                            # Detect SWR - Spindles coupling #
#######################################################################################

before = 1000 # Max distance in ms between a SWR and a spindle to be considered as Precoupled
after = 1000 # Max distance in ms between a spindle and a SWR to be considered as Postcoupled

listSpdlstarts=listSpdl["start time"]
listSpdlends=listSpdl["end time"]
listSWRstarts=listSWR["start time"]
listSWRends=listSWR["end time"]


TotcountSWR=len(listSWRstarts)
TotcountSpdl=len(listSpdlstarts)

TotcountSWRcoupled=0
TotcountSpdlcoupled=0

for ss in range(len(listSpdlstarts)): 
    startSpdl=listSpdlstarts[ss]
    endSpdl=listSpdlends[ss]

    startClosest_SWR = take_closest2(listSWRstarts, startSpdl)
    distance = startClosest_SWR - startSpdl
    if (distance > (- before)) and (distance <  0):
        Spdl_statut = 1
        TotcountSpdlcoupled+=1
    elif (distance > (0)) and (distance <  after):
        Spdl_statut = 1
        TotcountSpdlcoupled+=1
    else:
        Spdl_statut= 0

    listSpdl.loc[ss, 'Coupled to SWR?'] = Spdl_statut

print(Spdlfile)
writer = pd.ExcelWriter(Spdlfile)
listSpdl.to_excel(writer)
writer.close()

#######################################################

for ss in range(len(listSWRstarts)): 
    startSWR=listSWRstarts[ss]
    endSWR=listSWRstarts[ss]

    startClosest_Spdl = take_closest2(listSpdlstarts, startSWR)
    distance = startClosest_Spdl - startSWR
    if (distance > (- before)) and (distance <  0):
        SWR_statut = 1
        TotcountSWRcoupled+=1
    elif (distance > (0)) and (distance <  after):
        SWR_statut = 1
        TotcountSWRcoupled+=1
    else:
        SWR_statut= 0

    listSWR.loc[ss, 'Coupled to Spindles?'] =SWR_statut

print(SWRfile)
writer = pd.ExcelWriter(SWRfile)
listSWR.to_excel(writer)
writer.close()

data=[TotcountSpdl, TotcountSpdlcoupled, (TotcountSpdlcoupled/TotcountSpdl)*100, TotcountSWR, TotcountSWRcoupled, (TotcountSWRcoupled/TotcountSWR)*100]
GlobalResults= pd.DataFrame(data, columns=['Total Nb of Spdl', 'Total Nb of coupled Spdl', 'Ratio in percentage of coupled Spdl', 'Total Nb of SWR', 'Total Nb of coupled SWR', 'Ratio in percentage of coupled SWR'])
Globalfile=f'Oscillations_coupling_Statistics.xlsx'
writer = pd.ExcelWriter(Globalfile)
GlobalResults.to_excel(writer)
writer.close()